In [1]:
!hpc

connect to node gpu02 of hpc provided by BDSRC


In [2]:
from sqlalchemy import create_engine


engine = create_engine('mysql+pyodbc://telecom')
%load_ext sql
%sql engine
%config SqlMagic.autopandas = True

Found pyproject.toml from '/Users/jacky/github/telecom'

In [3]:
%sql SHOW tables

Running query in 'mysql+pyodbc://telecom'

10 rows affected.

,Tables_in_telecom
0,cinfo_xy
1,sample_sight_201308
2,sample_user_201308
3,serv_acct_item_0838_201308
4,serv_acct_item_0838_201406
5,tb_asz_cdma_0838_201308
6,tb_asz_cdma_0838_201406
7,tower
8,tower_center
9,tower_hot_201308


In [25]:
%%sql

DESC serv_acct_item_0838_201308

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,Field,Type,Null,Key,Default,Extra
0,SERV_ID,varchar(8),YES,MUL,None,
1,ACC_NBR,varchar(64),YES,,None,
2,TRAFFIC,int,YES,,None,
3,MONTH_NO,varchar(6),YES,,None,
4,CHARGE,int,YES,,None,
5,DURATION,int,YES,,None,
6,ACCT_ITEM_TYPE_CODE,varchar(6),YES,,None,
7,START_TIME,varchar(19),YES,,None,
8,END_TIME,varchar(19),YES,,None,
9,ETL_TYPE_ID,varchar(2),YES,,None,


# Calling Records

## Conclusion

- SERV_ID: user_id, can also be found in user information with the same column name
- ACC_NBR: cellphone number, can also be found in user infromation named as MSISDN
    - record the cellphone number of a user who is the client of the telecom company
- ACCT_ITEM_TYPE_CODE: 
- ETL_TYPE_ID: 
- CALLING_AREA_CODE, CALLED_AREA_CODE: determine the abstraction location of the calling and called phone call
- CALLING_NBR, CALLED_NBR: anonymous processed calling and called phone number
- LAC: local area code(most of them are outliers)
- CELL_ID: can be used for joing tower infromation on CELL16

- outlier
    - ACCT_ITEM_TYPE_CODE: -1
    - START_TIME, END_TIME: NOT LIKE '____/%/% %:__:__'
    - CALLING_AREA_CODE, CALLED_AREA_CODE: -1
    - CALLING_NBR, CALLED_NBR: NOT LIKE 'NotMobile_%'
    - LAC: -1, 0
    - CELL_ID: -1, 0

## Quickly look through the variables

In [5]:
%%sql

SELECT *
FROM serv_acct_item_0838_201308
LIMIT 5

Running query in 'mysql+pyodbc://telecom'

5 rows affected.

,SERV_ID,ACC_NBR,TRAFFIC,MONTH_NO,CHARGE,DURATION,ACCT_ITEM_TYPE_CODE,START_TIME,END_TIME,ETL_TYPE_ID,CALLED_AREA_CODE,CALLED_NBR,LAC,CELL_ID,RECV_TRAFFIC,CALLING_AREA_CODE,CALLING_NBR,SEND_TRAFFIC,BILLING_DURATION,CALLS
0,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:00:12,2013/8/1 0:00:12,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
1,a4zlm2z0,a1cnffy9,0,201308,12,1,020709,2013/8/1 0:03:03,2013/8/1 0:03:04,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
2,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:05:52,2013/8/1 0:05:52,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
3,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:08:44,2013/8/1 0:08:44,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1
4,a4zlm2z0,a1cnffy9,0,201308,12,0,020709,2013/8/1 0:11:45,2013/8/1 0:11:45,23,-1,NotMobile_-1,-1,-1,0,-1,NotMobile_-1,0,0,1


### Exception Detection

In [4]:
%%sql

SELECT
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE DURATION < 0

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,0


In [11]:
%%sql

SELECT 
    ACCT_ITEM_TYPE_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY  ACCT_ITEM_TYPE_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

48 rows affected.

,ACCT_ITEM_TYPE_CODE,COUNT(*)
0,170211,55828623
1,170209,43815325
2,160201,25888559
3,160101,24548848
4,160204,3484611
5,160501,3392784
6,170205,2594883
7,170202,2450229
8,170208,2262250
9,160603,2089841


In [14]:
%%sql

SELECT COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    START_TIME NOT LIKE '____/%/% %:__:__' 
    OR
    END_TIME NOT LIKE '____/%/% %:__:__'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,44849841


In [5]:
%%sql

SELECT
    ETL_TYPE_ID AS etl_type_id,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY etl_type_id

Running query in 'mysql+pyodbc://telecom'

8 rows affected.

,etl_type_id,COUNT(*)
0,21,60823665
1,22,60428727
2,23,4555058
3,24,43993499
4,31,50138
5,32,145182
6,33,6820
7,34,93054


In [2]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1
GROUP BY CALLING_AREA_CODE
ORDER BY COUNG(*) DESC

Running query in 'mysql+pyodbc://telecom'

510 rows affected.

,calling_area_code,count
0,0838,96205674
1,028,3371312
2,0816,682887
3,023,292693
4,010,155611
...,...,...
505,00961,1
506,00973,1
507,08029,1
508,08071,1


In [5]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1
GROUP BY CALLING_AREA_CODE
ORDER BY COUNG(*) DESC
LIMIT 20

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,CALLING_AREA_CODE,count
0,0838,96205674
1,028,3371312
2,0816,682887
3,023,292693
4,010,155611
5,021,155577
6,0817,127257
7,0755,110263
8,0825,106029
9,0833,99333


In [9]:
%%sql

SELECT
    CALLING_AREA_CODE,
    COUNT(*)
FROM serv_acct_item_0838_201308
WHERE CALLING_AREA_CODE != -1 AND LENGTH(CALLING_AREA_CODE) < 3
GROUP BY CALLING_AREA_CODE
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

""


In [11]:
%%sql

SELECT
    CALLED_AREA_CODE,
    COUNT(*) AS count
FROM serv_acct_item_0838_201308
WHERE CALLED_AREA_CODE != -1
GROUP BY CALLED_AREA_CODE
ORDER BY count DESC
LIMIT 20

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,CALLED_AREA_CODE,count
0,0838,93860363
1,028,3794338
2,0816,761832
3,023,320694
4,0817,147813
5,010,134660
6,0825,129196
7,021,119936
8,0833,107266
9,0755,101168


In [14]:
%%sql

SELECT
    CALLED_AREA_CODE,
    COUNT(*) AS count
FROM serv_acct_item_0838_201308
WHERE CALLED_AREA_CODE != -1 AND LENGTH(CALLED_AREA_CODE) < 3
GROUP BY CALLED_AREA_CODE
ORDER BY count DESC

Running query in 'mysql+pyodbc://telecom'

""


In [6]:
%%sql

SELECT
    LAC AS lac,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY lac

Running query in 'mysql+pyodbc://telecom'

115 rows affected.

,lac,COUNT(*)
0,-1,109329163
1,0,2336979
2,00000,46453
3,00001,214
4,00003,79
...,...,...
110,0264A,1
111,02A77,1
112,03133,1
113,09000,1


In [15]:
%%sql

SELECT
    CELL_ID,
    COUNT(*)
FROM serv_acct_item_0838_201308
GROUP BY CELL_ID

Running query in 'mysql+pyodbc://telecom'

57001 rows affected.

,CELL_ID,COUNT(*)
0,-1,48985582
1,0,2336698
2,0000,47155
3,0001,2184
4,0002,1151
...,...,...
56996,FFF2,32
56997,FFF3,2
56998,FFF7,249
56999,FFF9,2


In [17]:
%%sql

SELECT DISTINCT
    CELL_16
FROM cinfo_xy

Running query in 'mysql+pyodbc://telecom'

40569 rows affected.

,CELL_16
0,0001
1,0002
2,0003
3,0004
4,0005
...,...
40564,FFE8
40565,FFF0
40566,FFF2
40567,FFF4


In [18]:
%%sql

SELECT COUNT(*)
FROM cinfo_xy
WHERE LENGTH(CELL_16) < 2

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,0


###  try to figure out the period of the records

In [3]:
%%sql

SELECT DISTINCT
    MONTH_NO
FROM serv_acct_item_0838_201308

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,MONTH_NO
0,201308


In [6]:
%%sql 

SELECT
    date, 
    COUNT(*) AS count
FROM (
    SELECT SUBSTRING_INDEX(START_TIME,' ',1) AS date
    FROM serv_acct_item_0838_201308
) AS subquery
GROUP BY date

Running query in 'mysql+pyodbc://telecom'

31 rows affected.

,date,count
0,2013/8/1,6179713
1,2013/8/10,5070191
2,2013/8/11,5010373
3,2013/8/12,5501876
4,2013/8/13,5760491
5,2013/8/14,5413238
6,2013/8/15,5415015
7,2013/8/16,5758206
8,2013/8/17,5054504
9,2013/8/18,4982765


In [7]:
%%sql 

SELECT
    date, 
    COUNT(*) AS count
FROM (
    SELECT SUBSTRING_INDEX(START_TIME,' ',1) AS date
    FROM serv_acct_item_0838_201406
) AS subquery
GROUP BY date

Running query in 'mysql+pyodbc://telecom'

33 rows affected.

,date,count
0,2014/5/29,3
1,2014/5/30,1
2,2014/5/31,1396
3,2014/6/1,6079751
4,2014/6/10,5354899
5,2014/6/11,5387891
6,2014/6/12,5223904
7,2014/6/13,5434661
8,2014/6/14,5064052
9,2014/6/15,5159278


## Inspect the meaning of some column

In [3]:
%config SqlMagic.named_parameters=True

In [ ]:
%%sql

df << SELECT
    B.CI_BRANCH                                                                   AS branch,
    B.CI_DISTRICT                                                                 AS district,
    A.SERV_ID                                                                     AS id,
    A.ACC_NBR                                                                     AS client_nbr, 
    CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(A.END_TIME,' ',-1),':',1),UNSIGNED)   AS time,
    DAYOFWEEK(SUBSTRING_INDEX(A.END_TIME,' ',1))                                  AS day_of_week,
    A.CALLING_NBR                                                                 AS calling_nbr,
    A.CALLING_AREA_CODE                                                           AS calling_area_code,
    A.DURATION                                                                    AS duration,
    A.CALLED_AREA_CODE                                                            AS called_area_code,
    A.CALLED_NBR                                                                  AS CALLED_NBR, 
    A.CHARGE                                                                      AS charge, 
    A.ACCT_ITEM_TYPE_CODE                                                         AS acct_item_type_code, 
    A.ETL_TYPE_ID                                                                 AS etl_type_id,
    A.CELL_ID                                                                     AS cell_id
FROM serv_acct_item_0838_201308 AS A 
LEFT JOIN tb_asz_cdma_0838_201308 B ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
WHERE 
    A.ACCT_ITEM_TYPE_CODE != -1                                                    AND
    (A.START_TIME LIKE "____/%/% %:__:__" AND A.END_TIME LIKE "____/%/% %:__:__")  AND
    (A.CALLING_NBR NOT LIKE 'NotMobile_%' AND A.CALLED_NBR NOT LIKE 'NotMobile_%') AND
    (A.CALLING_AREA_COdE != '-1' AND A.CALLED_AREA_CODE != '-1')                   AND
    (A.LAC != '-1' AND A.LAC != '0')                                               AND
    (A.CELL_ID != '-1' AND A.CELL_ID != '0')

Running query in 'mysql+pyodbc://telecom'

53121025 rows affected.

In [ ]:
df.write_parquet('data/processed/cdr.parquet')

## Inspect the meaning of  Area Code
- [code information](https://en.wikipedia.org/wiki/Telephone_numbers_in_China)

- 0838: 四川德陽市
- 028: 成都
- 0816: 四川綿陽市
- 023: 重慶
- 010: 北京
- 021: 上海
- 0817: 四川南充市
- 0755: 廣東深圳市
- 0825: 四川遂寧市
- 0833: 四川樂山市
- 020: 廣州
- 029: 陝西西安、咸陽市
- 028: 成都、眉山、資陽(028M, 028Z)
- 0839: 四川廣元
- 0834: 四川涼山市
- 0832: 四川內江市(0832a)
- 0830: 四川瀘州市
- 0871: 雲南昆明市
- 0813: 四川自貢市

- 0831: 四川宜賓市
- 0835: 四川雅安市

In [2]:
%%sql

SELECT 
    A.SERV_ID, B.CI_DISTRICT, A.ACC_NBR, A.CALLING_NBR, A.CALLING_AREA_CODE, A.DURATION, A.CALLED_AREA_CODE, A.CALLED_NBR
FROM serv_acct_item_0838_201308 A 
LEFT JOIN tb_asz_cdma_0838_201308 B ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
WHERE 
    (CALLING_NBR != "NotMobile_%" AND CALLED_NBR != "NotMobile_%")
    AND
    (CALLING_AREA_COdE != -1 AND CALLED_AREA_CODE != -1)
LIMIT 20

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,SERV_ID,CI_DISTRICT,ACC_NBR,CALLING_NBR,CALLING_AREA_CODE,DURATION,CALLED_AREA_CODE,CALLED_NBR
0,9ndm2e3m,德阳现业,9pzhk02l,9pzhk02l,0838,0,028,8v9dgn8l
1,9ndm2e3m,德阳现业,9pzhk02l,8v9dgn8l,028,0,0838,9pzhk02l
2,9ndm2e3m,德阳现业,9pzhk02l,8v9dgn8l,028,0,0838,9pzhk02l
3,9ndm2e3m,德阳现业,9pzhk02l,8v9dgn8l,028,0,0838,9pzhk02l
4,9ndm2e3m,德阳现业,9pzhk02l,9pzhk02l,0838,0,0838,knbw2w9a
5,aawu43g1,广汉,a8z43msp,NotMobile_8610006,0838,0,0838,a8z43msp
6,aawu43g1,广汉,a8z43msp,NotMobile_8610006,0838,0,0838,a8z43msp
7,908b26do,广汉,8fotk0gg,irdg7wkx,028,0,0838,8fotk0gg
8,aawu43g1,广汉,a8z43msp,NotMobile_8610659821,0838,0,0838,a8z43msp
9,908b26do,广汉,8fotk0gg,8fotk0gg,0838,0,028,irdg7wkx


In [5]:
%%sql

SELECT *
FROM serv_acct_item_0838_201308
WHERE 
    (CALLING_AREA_CODE != '0838' AND CALLED_AREA_CODE != '0838')
    AND
    (CALLING_NBR NOT LIKE "NotMobile_%" AND CALLED_NBR NOT LIKE "NotMobile_%")
    AND
    (CALLING_AREA_COdE != -1 AND CALLED_AREA_CODE != -1)

Running query in 'mysql+pyodbc://telecom'

180 rows affected.

,SERV_ID,ACC_NBR,TRAFFIC,MONTH_NO,CHARGE,DURATION,ACCT_ITEM_TYPE_CODE,START_TIME,END_TIME,ETL_TYPE_ID,CALLED_AREA_CODE,CALLED_NBR,LAC,CELL_ID,RECV_TRAFFIC,CALLING_AREA_CODE,CALLING_NBR,SEND_TRAFFIC,BILLING_DURATION,CALLS
0,8mz3ifsu,a8zdjhjx,0,201308,598,105,160402,2013/8/1 12:42:55,2013/8/1 12:44:40,21,0031,kogl6ht6,-1,-1,0,0031,a8zdjhjx,0,0,1
1,8mz3ifsu,a8zdjhjx,0,201308,299,42,160402,2013/8/1 13:33:43,2013/8/1 13:34:25,21,0031,kogl6ht6,-1,-1,0,0031,a8zdjhjx,0,0,1
2,8mz3ifsu,a8zdjhjx,0,201308,198,71,160302,2013/8/1 13:59:29,2013/8/1 14:00:40,21,0031,a8zdjhjx,-1,-1,0,0031,kogl6ht6,0,0,1
3,8uoz4b05,71e02zhs,0,201308,99,13,160402,2013/8/3 15:49:31,2013/8/3 15:49:44,21,0758,i2hk2sed,-1,-1,0,0031,71e02zhs,0,0,1
4,8uaf6aj6,gcxm27y8,0,201308,1393,414,160402,2013/8/2 18:17:08,2013/8/2 18:24:02,21,020,iifvls71,-1,-1,0,0031,gcxm27y8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,9xcgdqfo,kjni2d1m,0,201308,299,19,160402,2013/8/29 20:47:06,2013/8/29 20:47:25,21,0031,9iki7uth,-1,-1,0,0031,kjni2d1m,0,0,1
176,9xcgdqfo,kjni2d1m,0,201308,299,47,160402,2013/8/29 20:22:10,2013/8/29 20:22:57,21,0031,9iki7uth,-1,-1,0,0031,kjni2d1m,0,0,1
177,9xcgdqfo,kjni2d1m,0,201308,299,39,160402,2013/8/29 21:02:46,2013/8/29 21:03:25,21,0031,9iki7uth,-1,-1,0,0031,kjni2d1m,0,0,1
178,9xcgdqfo,kjni2d1m,0,201308,299,17,160402,2013/8/29 19:44:56,2013/8/29 19:45:13,21,0031,lan44ne2,-1,-1,0,0031,kjni2d1m,0,0,1


## call within deyang

In [8]:
%%sql

SELECT COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    CALLING_AREA_CODE = '0838' AND CALLED_AREA_CODE = '0838'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,85122634


## call out from deyang

In [10]:
%%sql

SELECT COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    CALLING_AREA_CODE = '0838' AND CALLED_AREA_CODE != '0838'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,11083040


## call in to deyang

In [11]:
%%sql

SELECT COUNT(*)
FROM serv_acct_item_0838_201308
WHERE 
    CALLING_AREA_CODE != '0838' AND CALLED_AREA_CODE = '0838'

Running query in 'mysql+pyodbc://telecom'

1 rows affected.

,COUNT(*)
0,8737729


## Check the meaning of ACC_NRB

In [11]:
%%sql

SELECT ACC_NBR, CALLING_NBR, CALLING_AREA_CODE, DURATION, CALLED_AREA_CODE, CALLED_NBR
FROM serv_acct_item_0838_201308
WHERE 
    (CALLING_AREA_CODE = '0838' AND  CALLED_AREA_CODE != '0838') 
    AND
    ACC_NBR != CALLING_NBR
    AND
    (CALLING_NBR != "NotMobile_%" AND CALLED_NBR != "NotMobile_%")
    AND
    (CALLING_AREA_COdE != -1 AND CALLED_AREA_CODE != -1)

Running query in 'mysql+pyodbc://telecom'

184 rows affected.

,ACC_NBR,CALLING_NBR,CALLING_AREA_CODE,DURATION,CALLED_AREA_CODE,CALLED_NBR
0,2bf3fncu,l2sam7h2,0838,67,0031,2bf3fncu
1,2bf3fncu,gs93jfe8,0838,397,0031,2bf3fncu
2,ippu3hxt,53xwacjf,0838,9,0031,ippu3hxt
3,ippu3hxt,a91k4hnt,0838,101,0031,ippu3hxt
4,4gpk4hcn,md0kjm9j,0838,120,0031,4gpk4hcn
...,...,...,...,...,...,...
179,49844hqi,fu4s2k2c,0838,111,0031,49844hqi
180,49844hqi,g8xs4hqo,0838,21,0031,49844hqi
181,49844hqi,29pcagju,0838,33,0031,49844hqi
182,j9484hz1,kntk2bfa,0838,11,0031,j9484hz1


In [12]:
%%sql

SELECT ACC_NBR, CALLING_NBR, CALLING_AREA_CODE, DURATION, CALLED_AREA_CODE, CALLED_NBR
FROM serv_acct_item_0838_201308
WHERE 
    (CALLED_AREA_CODE = '0838' AND CALLING_AREA_CODE != '0838') 
    AND
    ACC_NBR != CALLED_NBR
    AND
    (CALLING_NBR != "NotMobile_%" AND CALLED_NBR != "NotMobile_%")
    AND
    (CALLING_AREA_COdE != -1 AND CALLED_AREA_CODE != -1)

Running query in 'mysql+pyodbc://telecom'

264 rows affected.

,ACC_NBR,CALLING_NBR,CALLING_AREA_CODE,DURATION,CALLED_AREA_CODE,CALLED_NBR
0,71e02zhs,71e02zhs,0031,271,0838,kuz2juae
1,71e02zhs,71e02zhs,0031,37,0838,fii4kxn0
2,71e02zhs,71e02zhs,0031,81,0838,fii4kxn0
3,ippu3hxt,ippu3hxt,0031,113,0838,l6na2sbm
4,ippu3hxt,ippu3hxt,0031,75,0838,jdfs67mi
...,...,...,...,...,...,...
259,5rbd2c6v,5rbd2c6v,0031,10,0838,get497hs
260,awi05rkm,awi05rkm,0031,158,0838,iapc6ekh
261,awi05rkm,awi05rkm,0031,242,0838,iapc6ekh
262,awi05rkm,awi05rkm,0031,353,0838,iapc6ekh
